In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')
# !cp "/content/drive/MyDrive/ma-vqc_datasets/Quartered anomaly size.zip" "/content"
# # Restart runtime before running rest of code
# !pip install open3d

Mounted at /content/drive


In [1]:
import os
import random

import pandas as pd
import open3d as o3d
import zipfile
import glob
from joblib import load, dump
import pathlib

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import numpy as np

import torch
import scipy.spatial.distance
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch import nn
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

random.seed = 42

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
# zip_name = "Quartered anomaly size"

# # Load random forest model model for set anomaly size
# !cp "/content/drive/MyDrive/MA-VQC Data/Quartered Anomaly Size/RandomForestModels/rf_point_50-50.joblib" "/content/"
# # Load best performing PointNet model
# !cp "/content/drive/MyDrive/MA-VQC Data/Quartered Anomaly Size/PointNetModels/PointNet (IFP)/PointNet_90-10_sample_rate_1024_batch_size_32_lr_0.0001_9.pth" "/content/"



# with zipfile.ZipFile(f"/content/{zip_name}.zip", 'r') as zip_ref: 
#     zip_ref.extractall("/content/")


# # Preprocessing step to change paths to Google Colab compatible paths

# # Correcting paths to be Google Colab compatible
# def preprocess_path(path, dataset_name):
#   idx = path.rindex(dataset_name) 
#   replaced = path[idx:].replace("\\", "/") 
#   return os.path.join("/content/", replaced)


# # Concatenate all dataset versions
# train_df = pd.concat([pd.read_csv(f"/content/{zip_name}/Var1/point_cloud_mesh/corrected_train_50-50_class_ration.csv"),
#                 pd.read_csv(f"/content/{zip_name}/Var2/point_cloud_mesh/corrected_train_50-50_class_ration.csv"),
#                 pd.read_csv(f"/content/{zip_name}/Var3/point_cloud_mesh/corrected_train_50-50_class_ration.csv"),
#                 pd.read_csv(f"/content/{zip_name}/Var4/point_cloud_mesh/corrected_train_50-50_class_ration.csv")])

# test_df = pd.concat([pd.read_csv(f"/content/{zip_name}/Var1/point_cloud_mesh/corrected_test_50-50_class_ration.csv"),
#                 pd.read_csv(f"/content/{zip_name}/Var2/point_cloud_mesh/corrected_test_50-50_class_ration.csv"),
#                 pd.read_csv(f"/content/{zip_name}/Var3/point_cloud_mesh/corrected_test_50-50_class_ration.csv"),
#                 pd.read_csv(f"/content/{zip_name}/Var4/point_cloud_mesh/corrected_test_50-50_class_ration.csv")])
# print(train_df.shape)
# print(test_df.shape)


# train_df["object_pcd_path"] = [preprocess_path(path, zip_name) for path in train_df["object_pcd_path"].values]
# train_df["object_path"] = [preprocess_path(path, zip_name) for path in train_df["object_path"].values]
# train_df["reference_path"] = [preprocess_path(path, zip_name) for path in train_df["reference_path"].values]

# test_df["object_pcd_path"] = [preprocess_path(path, zip_name) for path in test_df["object_pcd_path"].values]
# test_df["object_path"] = [preprocess_path(path, zip_name) for path in test_df["object_path"].values]
# test_df["reference_path"] = [preprocess_path(path, zip_name) for path in test_df["reference_path"].values]


# train_df.to_csv("/content/train_df.csv", index=False)
# test_df.to_csv("/content/test_df.csv", index=False)

In [ ]:
# Copy random forest model
# Load best performing PointNets
# !cp "/content/drive/MyDrive/MA-VQC Data/Halved Anomaly Size/PointNetModels/PointNet (RF)/PointNet_90-10_RF_samples_1024_batch_size_32_lr_0.0001_9.pth" "/content/"

In [4]:
# Load point model
rf_point = load("./model_weights/rf_point.joblib")

## PointNet Model

In [5]:
# Used for extracting feature rich vector giving a 1 dimensional vector for point net

class Tnet(nn.Module):
   def __init__(self, k=3):
      super().__init__()
      self.k=k
      self.conv1 = nn.Conv1d(k,64,1)
      self.conv2 = nn.Conv1d(64,128,1)
      self.conv3 = nn.Conv1d(128,1024,1)
      self.fc1 = nn.Linear(1024,512)
      self.fc2 = nn.Linear(512,256)
      self.fc3 = nn.Linear(256,k*k)

      self.bn1 = nn.BatchNorm1d(64)
      self.bn2 = nn.BatchNorm1d(128)
      self.bn3 = nn.BatchNorm1d(1024)
      self.bn4 = nn.BatchNorm1d(512)
      self.bn5 = nn.BatchNorm1d(256)
       

   def forward(self, input):
      # input.shape == (bs,n,3)
      bs = input.size(0)
      xb = F.relu(self.bn1(self.conv1(input)))
      xb = F.relu(self.bn2(self.conv2(xb)))
      xb = F.relu(self.bn3(self.conv3(xb)))
      pool = nn.MaxPool1d(xb.size(-1))(xb)
      flat = nn.Flatten(1)(pool)
      xb = F.relu(self.bn4(self.fc1(flat)))
      xb = F.relu(self.bn5(self.fc2(xb)))
      
      #initialize as identity
      init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
      if xb.is_cuda:
        init=init.cuda()
      matrix = self.fc3(xb).view(-1,self.k,self.k) + init
      return matrix

# Used for position estimation and point estimation using global and
# local coordinates
class Transform(nn.Module):
   def __init__(self):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)
        self.conv1 = nn.Conv1d(3,64,1)#(3,64,1)

        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)
       

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
       
   def forward(self, input):
        matrix3x3 = self.input_transform(input)
        # batch matrix multiplication
        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)

        xb = F.relu(self.bn1(self.conv1(xb)))

        matrix64x64 = self.feature_transform(xb)
        xb = torch.bmm(torch.transpose(xb,1,2), matrix64x64).transpose(1,2)

        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = self.bn3(self.conv3(xb))
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        output = nn.Flatten(1)(xb)
        return output, matrix3x3, matrix64x64

class PointNet(nn.Module):
    def __init__(self, classes = 2):
        super().__init__()
        self.transform = Transform()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, classes)
        

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        xb, matrix3x3, matrix64x64 = self.transform(input)
        xb1 = F.relu(self.bn1(self.fc1(xb)))
        xb2 = F.relu(self.bn2(self.dropout(self.fc2(xb1))))
        output = self.fc3(xb2)
        return self.logsoftmax(output), matrix3x3, matrix64x64, xb1, xb2 

In [6]:
# PointNet custom loss calculating with all 3 model outputs

# LOSS
def pointnetloss(outputs, labels, m3x3, m64x64, alpha = 0.0001):
    criterion = torch.nn.NLLLoss()
    bs = outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs, 1, 1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs, 1, 1)
    if outputs.is_cuda:
        id3x3 = id3x3.cuda()
        id64x64 = id64x64.cuda()
    diff3x3 = id3x3 - torch.bmm(m3x3, m3x3.transpose(1, 2))
    diff64x64 = id64x64 - torch.bmm(m64x64, m64x64.transpose(1, 2))
    return criterion(outputs, labels) + alpha * (torch.norm(diff3x3) + torch.norm(diff64x64)) / float(bs)

Data loader for PointNet

In [7]:
# Normalization used as a preprocessing step
class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0) 
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud

def default_transforms():
    return transforms.Compose([
                                Normalize(),
                                
                              ])

In [9]:
class PointCloudData(Dataset):
    def __init__(self, dataframe_path, valid=False, sample_rate=1024, transform=default_transforms()):
        # Get data
        self.df = pd.read_csv(dataframe_path)
        # class dict
        self.classes = {"anomaly": 1, "normal": 0}
        self.sample_rate=sample_rate
        self.transforms=transform

    def __len__(self):
        return self.df.shape[0]

    def __preproc__(self, file):
      # Cloud is loaded
        point_cloud = o3d.io.read_point_cloud(file)
        # Cloud is downsampled to selected size
        resampled = point_cloud.farthest_point_down_sample(num_samples=self.sample_rate)
        points = resampled.points
        np_array = np.array(points)
        if self.transforms:
            pointcloud = self.transforms(np_array)
        return torch.from_numpy(pointcloud)

    def __getitem__(self, idx):
        pcd_path = self.df.iloc[idx]['object_path']
        category = self.df.iloc[idx]['label']
        # with open(pcd_path, 'r') as f:
        pointcloud = self.__preproc__(pcd_path)
        return {'pointcloud': pointcloud, 
                'category': self.classes[category]}



In [11]:
batch_size = 32
learning_rate = 0.0001
sample_rate = 1024
epochs = 10

train_ds = PointCloudData("./data/train_df.csv", sample_rate=sample_rate)
test_ds = PointCloudData("./data/test_df.csv", sample_rate=sample_rate)

train_loader = DataLoader(dataset=train_ds, batch_size=batch_size)
test_loader = DataLoader(dataset=test_ds, batch_size=batch_size)

In [12]:


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

pointnet = PointNet()
pointnet.to(device)
optimizer = torch.optim.Adam(pointnet.parameters(), lr=learning_rate)

pointnet.load_state_dict(torch.load("./model_weights/PointNet_9.pth"))
pointnet.eval();

cpu


In [13]:
level_1_embs = []
level_2_embs = []
with torch.no_grad():
    for i, data in enumerate(train_loader):
        print('Batch [%4d / %4d]' % (i+1, len(train_loader)))
                   
        pcd = data['pointcloud'].to(device).float()
        labels = data['category'].to(device)
        optimizer.zero_grad()

        _, _, _, emb1, emb2 = pointnet(pcd.transpose(1,2))
        # _, preds = torch.max(outputs.data, 1)
        level_1_embs += list(emb1.cpu().numpy())
        level_2_embs += list(emb2.cpu().numpy())

lvl1_np = np.array(level_1_embs)
lvl2_np = np.array(level_2_embs)

Batch [   1 /    1]


In [14]:
# Create aggregated data
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

def is_anom(series):
  return series.max()


#Aggregate data for training the model to detect anomaly mesh
def aggregate_data(df):
  aggr_df = df.groupby(['id']).agg(
      {'label_obj': is_anom,#"first",
      'dist':[
          'mean',
          'max',
          percentile(60),
          percentile(70),
          percentile(80),
          percentile(90),
          percentile(95),
          percentile(96),
          percentile(97),
          percentile(98),
          percentile(99),
      ],
      'prob_anom':[
          'mean',  
          'max', 
          percentile(60),
          percentile(70),
          percentile(80),
          percentile(90),
          percentile(95),
          percentile(96),
          percentile(97),
          percentile(98),
          percentile(99),
      ],
      }
  )
  aggr_df.columns = ['_'.join(col) for col in aggr_df.columns.values]
  return aggr_df

def create_rf_point_samples(df, rf_point, sample_rate, name):
  full_aggr_df = None
  new_path_list = []
  for idx, (path, label) in enumerate(zip(df["reference_path"], df["label"])):
    tmp_df = pd.read_csv(path)
    tmp_df["id"] = idx
    tmp_df["label_obj"] = 1 if label == "anomaly" else 0
    preds = rf_point.predict_proba(tmp_df.loc[:, ["x", "y", "z", "dist"]].values)[:, 1]
    tmp_df["prob_anom"] = preds

    aggr = aggregate_data(tmp_df)
    if isinstance(full_aggr_df, type(None)):
      full_aggr_df = aggr
    else:
      full_aggr_df = pd.concat([full_aggr_df,
                              aggr], ignore_index=True)
      
  return full_aggr_df

In [ ]:
# !cp "/content/drive/MyDrive/MA-VQC Data/Quartered Anomaly Size/RandomForestData/aggregated_train.zip" "/content/"
# !cp "/content/drive/MyDrive/MA-VQC Data/Quartered Anomaly Size/RandomForestData/aggregated_test.zip" "/content/"


# with zipfile.ZipFile(f"/content/aggregated_train.zip", 'r') as zip_ref: 
#     zip_ref.extractall("")

# with zipfile.ZipFile(f"/content/aggregated_test.zip", 'r') as zip_ref: 
#     zip_ref.extractall("")


In [ ]:
# %%capture
# test_anomaly_mesh = create_rf_point_samples(test_df, rf_point, sample_rate, "test")
# train_anomaly_mesh = create_rf_point_samples(train_df, rf_point, sample_rate, "train")

In [15]:
train_anomaly_mesh = pd.read_csv("./data/aggregated_train.csv")
test_anomaly_mesh = pd.read_csv("./data/aggregated_test.csv")

In [16]:
full_train_dataset = []
for aggr_file, np_emb in zip(train_anomaly_mesh.iterrows(), lvl1_np):
  full_train_dataset.append(np.concatenate([aggr_file[1].drop(["label_obj_is_anom"]).values, np_emb]))

full_train_dataset_lvl2 = []
for aggr_file, np_emb in zip(train_anomaly_mesh.iterrows(), lvl2_np):
  full_train_dataset_lvl2.append(np.concatenate([aggr_file[1].drop(["label_obj_is_anom"]).values, np_emb]))


In [20]:
train_df = pd.read_csv("./data/train_df.csv")
test_df = pd.read_csv("./data/test_df.csv")

rf_mesh = RandomForestClassifier()
features = np.array(full_train_dataset)
labels = np.array([1 if val == "anomaly" else 0 for val in train_df['label']])
rf_mesh.fit(features, labels)

RandomForestClassifier()

In [21]:
test_level_1_embs = []
test_level_2_embs = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        print('Batch [%4d / %4d]' % (i+1, len(test_loader)))
                   
        pcd = data['pointcloud'].to(device).float()
        labels = data['category'].to(device)
        optimizer.zero_grad()

        _, _, _, emb1, emb2 = pointnet(pcd.transpose(1,2))
        # _, preds = torch.max(outputs.data, 1)
        test_level_1_embs += list(emb1.cpu().numpy())
        test_level_2_embs += list(emb2.cpu().numpy())

Batch [   1 /    1]


In [22]:
full_test_dataset = []
for aggr_file, np_emb in zip(test_anomaly_mesh.iterrows(), test_level_1_embs):
  full_test_dataset.append(np.concatenate([aggr_file[1].drop(["label_obj_is_anom"]).values, np_emb]))

full_test_dataset_lvl2 = []
for aggr_file, np_emb in zip(test_anomaly_mesh.iterrows(), test_level_2_embs):
  full_test_dataset_lvl2.append(np.concatenate([aggr_file[1].drop(["label_obj_is_anom"]).values, np_emb]))

preds = rf_mesh.predict(full_test_dataset)
test_labels = np.array([1 if val == "anomaly" else 0 for val in test_df['label']])
cm = confusion_matrix(test_labels, preds);
print(cm)

[[2 2]
 [1 3]]


In [23]:
lvl2_features = np.array(full_train_dataset_lvl2)
labels = np.array([1 if val == "anomaly" else 0 for val in train_df['label']])
rf_mesh_lvl2 = RandomForestClassifier()
rf_mesh_lvl2.fit(lvl2_features, labels)
preds_lvl2 = rf_mesh_lvl2.predict(full_test_dataset_lvl2)
cm = confusion_matrix(test_labels, preds_lvl2);
print(cm)

[[2 2]
 [4 0]]


In [24]:
from joblib import dump
dump(rf_mesh, "./model_weights/rf_mesh+PointNet_emb_512.joblib")
dump(rf_mesh_lvl2, "./model_weights/rf_mesh+PointNet_emb_256.joblib")
# !cp "/content/rf_mesh+PointNet_emb_512_50-50.joblib" "/content/drive/MyDrive/MA-VQC Data/Quartered Anomaly Size/RandomForestModels"
# !cp "/content/rf_mesh+PointNet_emb_256_50-50.joblib" "/content/drive/MyDrive/MA-VQC Data/Quartered Anomaly Size/RandomForestModels"

['./model_weights/rf_mesh+PointNet_emb_256.joblib']

90-10 split

In [ ]:
!cp "/content/drive/MyDrive/MA-VQC Data/Quartered Anomaly Size/RandomForestData/test_df_90_10.csv" "/content/"

In [25]:
def get_inds(df_90_10, full_df):
  inds = []
  for idx, filename in enumerate(full_df["object_path"]):
    if filename in df_90_10["object_path"].tolist():
      inds.append(idx)
  return inds

test_anomaly_mesh_90_10 = test_anomaly_mesh.iloc[get_inds(pd.read_csv("./data/test_df_90_10.csv"), test_df)]

In [26]:
# test_point_df_90_10["prob_anom"] = rf_point.predict_proba(test_point_df_90_10.loc[:, ['x', 'y', 'z', 'dist']].values)[:,1]
# test_anomaly_mesh_90_10 = aggregate_data(test_point_df_90_10)

test_dataset_90_10_lvl1 = []
test_dataset_90_10_lvl2 = []
for i in range(test_anomaly_mesh_90_10.shape[0]):
  idx = test_anomaly_mesh_90_10.index[i]
  test_dataset_90_10_lvl1.append(np.concatenate([test_anomaly_mesh_90_10.iloc[i].drop(["label_obj_is_anom"]).values, 
                                                 test_level_1_embs[idx]]))
  test_dataset_90_10_lvl2.append(np.concatenate([test_anomaly_mesh_90_10.iloc[i].drop(["label_obj_is_anom"]).values, 
                                                 test_level_2_embs[idx]]))

test_labels_90_10 = test_anomaly_mesh_90_10["label_obj_is_anom"].astype(int)
test_dataset_90_10_lvl1 = np.array(test_dataset_90_10_lvl1)
test_dataset_90_10_lvl2 = np.array(test_dataset_90_10_lvl2)

preds_90_10 = rf_mesh.predict(test_dataset_90_10_lvl1)
cm_90_10 = confusion_matrix(test_labels_90_10, preds_90_10);
print("90/10 split embedding 512")
print(cm_90_10)  

preds_lvl2_90_10 = rf_mesh_lvl2.predict(test_dataset_90_10_lvl2)
cm_lvl2_90_10 = confusion_matrix(test_labels_90_10, preds_lvl2_90_10);
print("90/10 split embedding 256")
print(cm_lvl2_90_10)

90/10 split embedding 512
[[2 2]
 [0 0]]
90/10 split embedding 256
[[2 2]
 [0 0]]
